In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext memory_profiler

In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.feature_processing import *
from src.item_processing import  *
from src.utils.general_utils import *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:

features_class = ItemFeatureProcessing(config)

In [ ]:
df_item = features_class.df_item
self = features_class

In [ ]:
pivot_table, index_col = self.get_clean_pivot_table('f__numeric_response',remove_low_freq_col=True)

In [ ]:
columns = []
for col in pivot_table.drop(columns = ['interview__id', 'roster_level', 'responsible']).columns:
    data = pivot_table[~pd.isnull(pivot_table[col])].copy()
    #anomalies = get_outlier_by_magnitude(data[col])
    #data = data[~anomalies]
    new_col = filter_columns_by_magnitude(data.drop(columns = ['interview__id', 'roster_level', 'responsible']),3).columns
    columns += list(new_col)
columns = list(set(columns))
columns

In [ ]:
# Extract the first digit from each number
for col in columns:
    df = pivot_table[(~pd.isnull(pivot_table[col]))&(pivot_table[col]>=1)].copy()
    df['first_digit'] = df[col].apply(lambda x: int(str(abs(x))[0]))
    # Calculate the distribution of first digits in the dataset
    actual_dist = df['first_digit'].value_counts(normalize=True).sort_index()
    
    # Calculate the expected distribution according to Benford's Law
    digits = range(1, 10)
    expected_dist = [np.log10(1 + 1/d) for d in digits]
    expected_series = pd.Series(expected_dist, index=digits)
    
    # Plot the actual vs expected distributions
    plt.bar(actual_dist.index, actual_dist.values, alpha=0.7, label='Actual')
    plt.bar(expected_series.index, expected_series.values, alpha=0.4, label='Expected (Benford)')
    plt.xlabel('Leading Digit')
    plt.ylabel('Proportion')
    plt.title(col)
    plt.legend()
    plt.show()

In [ ]:
pivot_table[['responsible']+columns]

In [ ]:
col = 'plot_area'
results_df = apply_benford_tests(pivot_table[['responsible']+columns], 'responsible', col, apply_first_digit=False)
results_df[col+'anomaly'] = results_df['p-value'].apply(lambda x: x<=0.05)
results_df#[col+'anomaly']

In [ ]:
import pandas as pd
import numpy as np




# Apply on our dataframe
col = columns[0]
data = pivot_table[~pd.isnull(pivot_table[col])&(pivot_table[col]>=1)].copy()
# Remove extreme outliers by order of magnitude
extreme_outliers = get_outlier_by_magnitude(data[col], mode_deviation=3)
data = data[~extreme_outliers].copy()
data['box_cox'] = get_box_cox_rescaled(data[col])
# remove oyutliers by zeta score on normalized data
lower_limit, upper_limit = get_outlier_z_score(data, 'box_cox', 2.5)
data = data[(~lower_limit)&(~upper_limit)].copy()  

results_df = apply_benford_tests(data[['responsible']+columns], 'responsible', col)
results_df['p-value'] = results_df['p-value'].apply(lambda x: x<=0.05)

results_df
#results_df['Chi2_p'].value_counts(), results_df['Fisher_p'].value_counts()



In [ ]:
col

In [ ]:
pivot_table['anomaly_count'].value_counts()

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chisquare

def benfords_law_for_value(value):
    # Calculate expected counts for each leading digit according to Benford's Law
    expected_counts = [np.log10(1 + 1/d) for d in range(1, 10)]

    # Extract leading digit and calculate its observed frequency
    leading_digit = int(str(value)[0])
    if 1 <= leading_digit <= 9:
        observed_freq = [1 if d == leading_digit else 0 for d in range(1, 10)]
    else:
        return 0  # Not a valid leading digit

    # Chi-squared test
    chi2_stat, p_val = chisquare(observed_freq, expected_counts)

    # If p-value is below a certain significance level (e.g., 0.05), 
    # then we can reject the null hypothesis that the observed counts 
    # follow the expected distribution.
    return p_val < 0.05

def count_anomalies_for_row(row):
    return sum([benfords_law_for_value(val) for val in row])

df = pd.DataFrame({
    'col1': [123, 456, 789, 101, 111, 555, 444],
    'col2': [100, 200, 300, 1000, 2000, 3000, 4000],
    'col3': [12, 23, 34, 45, 56, 67, 78]
})

# Count anomalies for each row
df['anomalies_count'] = df.apply(count_anomalies_for_row, axis=1)
print(df)
